In [1]:
import pickle
from attention import *
from torch.utils.data import DataLoader
from pytorch_util import RAdam,trainable_parameter
from torch.optim.lr_scheduler import ReduceLROnPlateau
#from pytorch_transformers import AdamW,WarmupLinearSchedule

In [39]:
batch_size = 1024
dim = 512
head_d = 8
head = SimplyInteraction #SimplyHead
encoder_layer,decoder_layer = 2,2
epochs = 10
clip = 0.1
logLoss = False
EncoderLayer=TransformerEncoderLayer
DecoderLayer=TransformerDecoderLayer

dim_feedforward = 1024
dropout = 0.05
lr = 3e-3

In [3]:
with open('../Data/train_data_attention_node.pickle', 'rb') as handle:
    train_node = pickle.load(handle)
with open('../Data/train_data_attention_edge.pickle', 'rb') as handle:
    train_edge = pickle.load(handle)
with open('../Data/train_data_attention_edge_y.pickle', 'rb') as handle:
    train_y = pickle.load(handle)
with open('../Data/train_data_ind.pickle', 'rb') as handle:
    ind = pickle.load(handle)

In [33]:
train_dl = attentionDataset(train_node[:4000000],train_edge[:4000000],ind[:4000000],train_y[:4000000])
train_dl = DataLoader(train_dl,shuffle=True,batch_size=batch_size,collate_fn=collate_fn,num_workers=4)
val_dl = attentionDataset(train_node[4000000:],train_edge[4000000:],ind[4000000:],train_y[4000000:])
val_dl = DataLoader(val_dl,shuffle=True,batch_size=batch_size,collate_fn=collate_fn,num_workers=4)

In [40]:
model=Attention(dim,encoder_layer,decoder_layer,head_d,head,EncoderLayer,DecoderLayer,
                dropout=dropout,dim_feedforward=dim_feedforward).to('cuda')
paras = trainable_parameter(model)
opt = RAdam(paras,lr=lr,weight_decay=1e-2)
#opt = AdamW(paras, lr=lr, correct_bias=False)
scheduler = ReduceLROnPlateau(opt, 'min',factor=0.5,patience=5,min_lr=1e-04)
#scheduler = WarmupLinearSchedule(opt, warmup_steps=40, t_total=100)

In [12]:
# out,mask,edge,y = next(iter(train_dl))
# out,mask,edge,y = out.to('cuda:0'),mask.to('cuda:0'),edge.to('cuda:0'),y.to('cuda:0')

# opt.zero_grad()
# loss,loss_perType = model(out,mask,edge,y,logLoss)
# loss.backward()

# [(n,p.std()) for n,p in model.named_parameters()]

# [(n,p.grad.mean()) for n,p in model.named_parameters()]

In [41]:
model,bestOpt,bestWeight = train_type(opt,model,epochs,train_dl,val_dl,paras,clip,\
                                       scheduler=scheduler,logLoss=logLoss)

epoch:0, train_loss: +4.873, val_loss: +1.229, 
train_vector: +2.59|+2.32|+0.97|+0.99|+0.98|+0.90|+1.07|-0.10, 
val_vector  : +2.44|+2.26|+0.92|+0.80|+1.33|+0.87|+1.17|+0.05

epoch:1, train_loss: +4.529, val_loss: +1.299, 
train_vector: +2.50|+2.26|+0.97|+0.92|+0.97|+0.89|+1.06|-0.16, 
val_vector  : +2.67|+2.29|+0.92|+1.10|+1.32|+0.86|+1.18|+0.06

epoch:2, train_loss: +4.514, val_loss: +1.239, 
train_vector: +2.50|+2.26|+0.97|+0.92|+0.97|+0.89|+1.06|-0.16, 
val_vector  : +2.34|+2.49|+0.92|+0.77|+1.32|+0.84|+1.17|+0.06



KeyboardInterrupt: 

In [43]:
a = torch.rand((3,4,5))

In [44]:
a

tensor([[[0.6116, 0.3827, 0.2909, 0.7461, 0.9686],
         [0.3556, 0.9787, 0.1412, 0.7747, 0.2204],
         [0.4146, 0.6009, 0.1438, 0.2266, 0.5505],
         [0.9805, 0.9418, 0.1804, 0.8635, 0.4107]],

        [[0.1023, 0.9522, 0.9565, 0.6346, 0.4059],
         [0.7043, 0.2908, 0.6731, 0.1355, 0.2348],
         [0.6275, 0.3243, 0.2004, 0.3885, 0.8659],
         [0.6583, 0.5330, 0.9002, 0.4966, 0.6999]],

        [[0.6217, 0.1492, 0.1007, 0.0670, 0.0799],
         [0.5098, 0.7673, 0.3892, 0.0952, 0.6599],
         [0.4721, 0.4205, 0.3383, 0.5202, 0.7436],
         [0.7209, 0.2704, 0.3852, 0.5210, 0.7989]]])

In [45]:
b = torch.randint(0,4,(3,2))

In [46]:
b

tensor([[0, 1],
        [2, 0],
        [2, 0]])

In [48]:
a[b].shape

torch.Size([3, 2, 4, 5])